In [32]:
#Import needed libraries

import csv 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon 
from shapely.geometry import Point
from shapely.geometry import shape
import folium 
from folium.plugins import HeatMap
import math
from pyproj import Proj


In [2]:
#Import Data

zones = pd.read_csv('zones.csv', sep=',')
yellow_trips = pd.read_csv('yellow_trips_new.csv', sep=',')
weather = pd.read_csv('weather.csv', sep=',')
uber_trips_2014 = pd.read_csv('uber_trips_2014.csv', sep=',')
uber_trips_2015 = pd.read_csv('uber_trips_2015.csv', sep=',')
mta_trips = pd.read_csv('mta_trips.csv', sep=',')
green_trips = pd.read_csv('green_trips_new_2.csv', sep=',')
geographic = pd.read_csv('geographic.csv', sep=',')
demographics = pd.read_csv('demographics.csv', sep=',')

C:\Users\MERQUEO\.conda\envs\ds4a_test\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#Check Geo

geographic.head()

,BK88,QN52,QN48,QN51,QN27,BX35,BX98,QN07,MN06,QN02,...,MN25,MN24,MN23,MN13,MN15,MN32,MN33,MN99,QN18,QN29
0,-73.976051,-73.794934,-73.775740,-73.803792,-73.861099,-73.896967,-73.872873,-73.757258,-73.946080,-73.760316,...,-74.000784,-73.992605,-73.989904,-74.001535,-73.993833,-73.938048,-73.935054,-74.010930,-73.858726,-73.867911
1,40.631284,40.757806,40.743334,40.775618,40.763673,40.834350,40.785983,40.718147,40.821271,40.675118,...,40.694295,40.724145,40.734443,40.762653,40.772940,40.780838,40.791695,40.684500,40.735717,40.744942
2,-73.977167,-73.794749,-73.775799,-73.800991,-73.859931,-73.896787,-73.872886,-73.755890,-73.946404,-73.758806,...,-74.000960,-73.992634,-73.989870,-74.001339,-73.993789,-73.937903,-73.935108,-74.011934,-73.858672,-73.867877
3,40.630755,40.755636,40.743326,40.775397,40.762209,40.834133,40.785971,40.716307,40.820580,40.673014,...,40.694077,40.724065,40.733535,40.762217,40.772912,40.780758,40.791658,40.683896,40.735627,40.744824
4,-73.977000,-73.794579,-73.778060,-73.798653,-73.859668,-73.896303,-73.872983,-73.754702,-73.946720,-73.758598,...,-74.003015,-73.993093,-73.990028,-74.001279,-73.993727,-73.937796,-73.935173,-74.012177,-73.858585,-73.867852


In [4]:
#Test: Lat, Long info 

test = list(geographic['BX10'])

long = [test[x] for x in range(0,len(test),1) if x%2 == 0 and math.isnan(test[x]) == False]
lat = [test[x] for x in range(0,len(test),1) if x%2 == 1 and math.isnan(test[x]) == False]

polygon_geom = Polygon(zip(long, lat))
crs = {'init': 'epsg:4326'}

polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
print(polygon.geometry)

polygon.to_file(filename='test_polygon.geojson', driver='GeoJSON')
polygon.to_file(filename='test_polygon.shp', driver="ESRI Shapefile")

json = polygon.to_json()

0    POLYGON ((-73.78833 40.83468, -73.78931 40.834...
Name: geometry, dtype: geometry


In [5]:
#Painting map

import folium 

m = folium.Map(
    location=[40.730610, -73.935242],
    tiles='cartodbpositron',
    zoom_start = 10  # Limited levels of zoom for free Mapbox tiles.
)

folium.GeoJson(
    json,
    name='test'
).add_to(m)

m


In [39]:
#Mapping all geo zones in the map (As polygons)
import json
geographic = pd.read_csv('geographic.csv', sep=',')

names = geographic.columns

loc_lists = list()
json_list = list()

lats = list()
longs = list()
polygons = []

var_lists = list()
var_lat = 0
var_long = 0

crs = {'init': 'epsg:4326'}

geos = []

for i in range(0,len(names),1):
    #var_list = list(geographic[names[i]]) 
    var_long = [var_list[x] for x in range(0,len(var_list),1) if x%2==0 and math.isnan(var_list[x]) == False ]
    var_lat = [var_list[x] for x in range(0,len(var_list),1) if x%2==1 and math.isnan(var_list[x]) == False ]
    polygon_geom = Polygon(zip(var_long, var_lat))
    polygons.append(polygons)
    properties = {
        'zone':names[i]
    }
    poly = {
        'type': 'Polygon',
        'coordinates': [[lon,lat] for lon,lat in zip(var_long,var_lat) ]
    }
    feature_content = {
        'type': 'Feature', 
        'properties' : properties, 
        'geometry':poly
    }
    geos.append(feature_content)
 
geometries = {
    'type': 'FeatureCollection',
    'features': geos,
}

geo_str = json.dumps(geometries) 

with open('geonyc.geojson', 'w') as outfile:
    json.dump(geo_str, outfile)

#nygeo = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygons])

#nygeo

#polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
#    json = polygon.to_json()
#    print(json)
#    loc_lists.append( list(geographic[names[i]]) )
#    json_list.append(json)


In [52]:
# load GeoJSON file containing sectors
with open('geonyc.geojson') as f:
    js = json.load(f)

# construct point based on lon/lat returned by geocoder
#point = Point(-122.7924463, 45.4519896)
point = Point(-73.867900, 40.74486)

# check each polygon to see if it contains the point
for feature in js['features']:
    #print(feature['geometry']['coordinates'])
    #polygon = shape(feature['geometry']['coordinates'])
    polygon = Polygon(zip(var_long, var_lat))
    #print(polygon)
    if polygon.contains(point):
        print('Found containing polygon:', feature['properties']['zone'])

Found containing polygon: BK88
Found containing polygon: QN52
Found containing polygon: QN48
Found containing polygon: QN51
Found containing polygon: QN27
Found containing polygon: BX35
Found containing polygon: BX98
Found containing polygon: QN07
Found containing polygon: MN06
Found containing polygon: QN02
Found containing polygon: QN46
Found containing polygon: BK25
Found containing polygon: BX59
Found containing polygon: QN41
Found containing polygon: MN17
Found containing polygon: MN19
Found containing polygon: QN47
Found containing polygon: QN08
Found containing polygon: QN54
Found containing polygon: QN25
Found containing polygon: QN55
Found containing polygon: QN06
Found containing polygon: BK44
Found containing polygon: BK40
Found containing polygon: BK41
Found containing polygon: BK95
Found containing polygon: BX33
Found containing polygon: BK69
Found containing polygon: BX52
Found containing polygon: QN49
Found containing polygon: QN26
Found containing polygon: BK27
Found co

In [47]:
json_list

[]

In [205]:
#Graph everything w/ geojson files:

import folium 

#Mapa Inicialización
m = folium.Map(
    location=[40.730610, -73.935242],
    tiles='cartodbpositron',
    zoom_start = 15  # Limited levels of zoom for free Mapbox tiles.
)

mitad = len(json_list)/3

for i in range(0,len(json_list),1):
    folium.GeoJson(
        json_list[i],
        name=names[i]
    ).add_to(m)



In [207]:
#Saving the .html file

m.save('map.html')

In [18]:
###################################################################################################
## Real Test
## Objetivos: Dejar el dataframe de ubicaciones usable para buscar cada punto en un NTA específico
###################################################################################################

#Eje de Coordenadas:
crs_data = {'init': 'epsg:4326'}

#Estructura del DataFrame Geo para tener la información de Polígonos
geo_names = geographic.columns

loc_lists = list()
json_list = list()
gdf_list = list()
lats = list()
longs = list()
var_lists = list()
var_lat = 0
var_long = 0
polygon_list = list()

for i in range(0,len(geo_names),1):
    var_list = list(geographic[geo_names[i]]) 
    var_long = [var_list[x] for x in range(0,len(var_list),1) if x%2==0 and math.isnan(var_list[x]) == False ]
    var_lat = [var_list[x] for x in range(0,len(var_list),1) if x%2==1 and math.isnan(var_list[x]) == False ]
    polygon_geom = Polygon(zip(var_long, var_lat))
    polygon_list.append(polygon_geom)

len(polygon_list)

195

In [288]:
data = {'nta_code': geo_names, 'polygon': polygon_list}

In [289]:
gdf_nta_test = gpd.GeoDataFrame( data=data, crs=crs, geometry=polygon_list )
gdf_nta_test.head()

,nta_code,polygon,geometry
0,BK88,"POLYGON ((-73.9760507905698 40.6312841471042, ...","POLYGON ((-73.97605 40.63128, -73.97717 40.630..."
1,QN52,"POLYGON ((-73.79493384594591 40.7578063026709,...","POLYGON ((-73.79493 40.75781, -73.79475 40.755..."
2,QN48,"POLYGON ((-73.7757397458527 40.7433339175923, ...","POLYGON ((-73.77574 40.74333, -73.77580 40.743..."
3,QN51,"POLYGON ((-73.8037916164017 40.7756183875692, ...","POLYGON ((-73.80379 40.77562, -73.80099 40.775..."
4,QN27,"POLYGON ((-73.86109864852619 40.7636727485249,...","POLYGON ((-73.86110 40.76367, -73.85993 40.762..."


In [293]:
gdf_nta = gpd.GeoDataFrame( data=data , crs=crs, geometry=data['polygon'] )

(195, 3)

In [187]:
ut_2014_df = uber_trips_2014.copy()
muestra = ut_2014_df[10000:11000]
muestra = muestra.reset_index(drop=True)

#Assign NTA to any point (lat, long) in the Data Frame:

In [264]:
##V2: sjoin (Spatial Join)

muestra = ut_2014_df.copy()

In [316]:
#### Pipeline completo:

#Manipulación Geo, básicas para ordenarse.
def to_points(lst_long,lst_lat):
    points = [Point(x) for x in (zip(lst_long,lst_lat))]
    return points

def to_geodf(df,active_geom, crs):
    return gpd.GeoDataFrame(data=df, crs=crs, geometry=active_geom)


In [ ]:
muestra['pickup_points'] = to_points(muestra['pickup_longitude'],muestra['pickup_latitude'])
geo_muestra = to_geodf(muestra,muestra['pickup_points'],crs_data)
geo_muestra.head()

In [299]:
%%time

geo_muestra_located = gpd.sjoin(geo_muestra,gdf_nta[['nta_code','polygon','geometry']],how='left',op='within')

Wall time: 1min 55s


In [318]:
geo_muestra_located = to_geodf(geo_muestra_located,'pickup_points',crs_data)

geo_muestra_located.geometry

0          POINT (-73.95490 40.76900)
1          POINT (-74.03450 40.72670)
2          POINT (-73.98730 40.73160)
3          POINT (-73.97760 40.75880)
4          POINT (-73.97220 40.75940)
                      ...            
4534322    POINT (-73.98450 40.76680)
4534323    POINT (-74.17730 40.69110)
4534324    POINT (-73.93190 40.85190)
4534325    POINT (-74.00660 40.70810)
4534326    POINT (-73.94960 40.71400)
Name: pickup_points, Length: 4534327, dtype: geometry

In [56]:
# Locating each travel points into the map: 

# yellow_trips 
# weather 
# uber_trips_2014 
# uber_trips_2015 
# mta_trips 
# green_trips 

folium_map = folium.Map(
            location=[40.730610, -73.935242],
            tiles='cartodbpositron',
            zoom_start = 12  # Limited levels of zoom for free Mapbox tiles.
        )

# uber_trips_2014.head()
# yellow_trips.head()
# mta_trips.head()
# green_trips.head()


muestra = uber_trips_2014.sample(n=1000, random_state=1)
for i in range(0,len(muestra),1):
    marker = folium.CircleMarker(location=[list(muestra["pickup_latitude"])[i],list(muestra["pickup_longitude"])[i]],radius=2, color="black",fill=True)
    marker.add_to(folium_map)

muestra_yellow = yellow_trips.sample(n=1000,random_state=1)
for i in range(0,len(muestra_yellow),1):
    marker = folium.CircleMarker(location=[list(muestra_yellow["pickup_latitude"])[i],list(muestra_yellow["pickup_longitude"])[i]],radius=2, color="yellow",fill=True)
    marker.add_to(folium_map)

muestra_green = green_trips.sample(n=1000,random_state=1)
for i in range(0,len(muestra_green),1):
    marker = folium.CircleMarker(location=[list(muestra_green["pickup_latitude"])[i],list(muestra_green["pickup_longitude"])[i]],radius=2, color="green",fill=True)
    marker.add_to(folium_map)

muestra_mta = mta_trips.sample(n=1000,random_state=1)
for i in range(0,len(muestra_green),1):
    marker = folium.CircleMarker(location=[list(muestra_mta["latitude"])[i],list(muestra_mta["longitude"])[i]],radius=2, color="red",fill=True)
    marker.add_to(folium_map)    
    
folium_map.save('map_comparison_samples.html')

In [75]:
#Heatmap for mta: just for fun!


folium_map = folium.Map(
            location=[40.730610, -73.935242],
            tiles='cartodbpositron',
            zoom_start = 10  # Limited levels of zoom for free Mapbox tiles.
        )

muestra_mta =mta_trips.sample(n=100000,random_state=1)
max_amount = max(muestra_mta.new_entries)
print(max_amount)

min_amount = min(muestra_mta.new_entries)
print(min_amount)


hm_wide = HeatMap( list(zip(muestra_mta['latitude'], muestra_mta['longitude'], muestra_mta['new_entries'])),
                   min_opacity=0.5,
                   max_val=max_amount,
                   radius=10, blur=5, 
                   max_zoom=20, 
                 )

folium_map.add_child(hm_wide)

folium_map.save('heatMapMTA100k.html')

1564
1


In [87]:
# How to plot a heatmap for the uber, boro and medallion taxi info? (Considering single points)


folium_map = folium.Map(
            location=[40.730610, -73.935242],
            tiles='cartodbpositron',
            zoom_start = 10  # Limited levels of zoom for free Mapbox tiles.
        )
#Uber 2014
muestra_uber_2014 =uber_trips_2014.sample(n=100000,random_state=1)
muestra_uber_2014["count"] = 1 #Count variable for uber data (2014)

hm_wide = HeatMap( list(zip(muestra_uber_2014['pickup_latitude'], muestra_uber_2014['pickup_longitude'], muestra_uber_2014['count'])),
                   min_opacity=0.5,
                   radius=5, blur=3, 
                   max_zoom=20, 
                 )

folium_map.add_child(hm_wide)

folium_map.save('heatMapUBER2014100k.html')

In [92]:

folium_map = folium.Map(
            location=[40.730610, -73.935242],
            tiles='cartodbpositron',
            zoom_start = 10  # Limited levels of zoom for free Mapbox tiles.
        )
#Green Trips
muestra_green = green_trips.sample(n=100000,random_state=1)
muestra_green["count"] = 1 #Count variable for uber data (2014)

muestra_green.head()

hm_wide = HeatMap( list(zip(muestra_green['pickup_latitude'], muestra_green['pickup_longitude'], muestra_green['count'])),
                   min_opacity=0.5,
                   radius=5, blur=3, 
                   max_zoom=20, 
                 )

folium_map.add_child(hm_wide)

folium_map.save('heatMapGreen100k.html')

In [93]:

folium_map = folium.Map(
            location=[40.730610, -73.935242],
            tiles='cartodbpositron',
            zoom_start = 10  # Limited levels of zoom for free Mapbox tiles.
        )
#Yellow Trips
muestra_yellow = yellow_trips.sample(n=100000,random_state=1)
muestra_yellow["count"] = 1 #Count variable for uber data (2014)

muestra_green.head()

hm_wide = HeatMap( list(zip(muestra_yellow['pickup_latitude'], muestra_yellow['pickup_longitude'], muestra_yellow['count'])),
                   min_opacity=0.5,
                   radius=5, blur=3, 
                   max_zoom=20, 
                 )

folium_map.add_child(hm_wide)

folium_map.save('heatMapYellow100k.html')